In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
from datetime import datetime, timedelta
import time
from dateutil import parser
plt.style.use('seaborn')
import seaborn as sns
from os import getenv
import pandas_ta as ta
import warnings
warnings.filterwarnings('ignore')

C:\Users\baruc\AppData\Local\Temp\ipykernel_13372\3046771581.py:10: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [55]:
api_key = getenv('api_key')
secret_key = getenv('api_secret')

In [56]:
client = Client(api_key = api_key, api_secret= secret_key, tld= 'com')

In [57]:
client

In [58]:
symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_5MINUTE
start_date = '2023-01-01'
end_date = '2024-11-02'

In [59]:
data = client.get_historical_klines(symbol, interval, start_str=start_date,end_str=end_date)

In [60]:
df = pd.DataFrame(data, columns=[
    'timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
    'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume', 'ignore'
])

In [61]:
df

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,1672531200000,16541.77000000,16544.76000000,16527.51000000,16535.38000000,486.60903000,1672531499999,8046638.38144730,14258,220.13975000,3640277.88516140,0
1,1672531500000,16534.91000000,16540.43000000,16522.55000000,16526.67000000,391.19043000,1672531799999,6467462.43432510,11944,189.61441000,3134879.21735500,0
2,1672531800000,16526.67000000,16530.87000000,16520.00000000,16520.69000000,294.73889000,1672532099999,4870445.53737180,11282,143.59637000,2372892.39973320,0
3,1672532100000,16521.26000000,16537.73000000,16517.72000000,16534.94000000,481.18777000,1672532399999,7951477.58749230,14060,260.25307000,4300613.86849020,0
4,1672532400000,16534.94000000,16540.66000000,16532.33000000,16535.54000000,309.53189000,1672532699999,5118776.84812870,10050,162.28132000,2683690.89180420,0
...,...,...,...,...,...,...,...,...,...,...,...,...
193228,1730504400000,69485.12000000,69515.16000000,69479.90000000,69489.83000000,27.34582000,1730504699999,1900362.15618040,3661,12.64316000,878626.75290750,0
193229,1730504700000,69489.83000000,69499.99000000,69472.00000000,69484.00000000,36.44869000,1730504999999,2532626.90660330,2645,6.10643000,424298.04732880,0
193230,1730505000000,69484.01000000,69560.00000000,69478.00000000,69540.00000000,33.85343000,1730505299999,2353469.49199610,4229,23.01662000,1600055.06261070,0
193231,1730505300000,69539.99000000,69546.00000000,69496.00000000,69496.01000000,10.94950000,1730505599999,761246.01934460,1901,4.05392000,281857.02950080,0


In [62]:
#Data Manipulation
# Convert the 'timestamp' column to a readable date format
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

In [63]:
# Select relevant columns
df = df[['open', 'high', 'low', 'close', 'volume']]

In [64]:
# Convert data types to numeric for analysis
df = df.apply(pd.to_numeric)

<p><strong>Preparación de datos</strong>:</p><ul><li>Calcule los promedios móviles (MA) para 5, 30 y 90 días utilizando tanto el promedio móvil simple (SMA) como el promedio móvil exponencial (EMA).</li><li>Identifique los eventos de "cruce dorado" y "cruce de la muerte" donde la MA de 5 días cruza la MA de 30 o 90 días.</li><li>Calcule los promedios de volumen y otros indicadores de volumen para respaldar el análisis de tendencias.</li></ul><p><strong>Ingeniería de características</strong>:</p><ul><li>Cree características que capturen los cruces de MA (cruce dorado, cruce de la muerte), las tendencias de volumen y otros indicadores estadísticos como el impulso relativo o la volatilidad.</li><li>Agregue etiquetas para la dirección de la tendencia (<code>hacia arriba</code>, <code>hacia abajo</code> o <code>hacia los lados</code>) y la acción recomendada (<code>comprar</code>, <code>vender</code> o <code>avanzar</code>). <code>hold</code>) en función de eventos de cruce y tendencias de volumen.</li><li>Defina la cantidad a negociar (<code>all</code>, <code>some</code> o <code>none</code>) en función de la fuerza de la señal.</li></ul><p><strong>Modelado con XGBoost:</strong></p><ul><li>Use XGBoost para entrenar un modelo que aprenda patrones en el comportamiento de precios y volumen, combinando señales de cruce históricas y tendencias de volumen para pronosticar la próxima acción.</li><li>Configure un marco de evaluación para optimizar el rendimiento del modelo para la clasificación de tendencia, acción y cantidad.</li></ul><p><br></p>

In [87]:
#Importando las librerias requeridas por el modelo. 
import xgboost as xgb
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [66]:
# Encode the target labels
le_trend = LabelEncoder()
le_action = LabelEncoder()
le_amount = LabelEncoder()

In [67]:
dataset = df

In [68]:
dataset

,open,high,low,close,volume
timestamp,,,,,
2023-01-01 00:00:00,16541.77,16544.76,16527.51,16535.38,486.60903
2023-01-01 00:05:00,16534.91,16540.43,16522.55,16526.67,391.19043
2023-01-01 00:10:00,16526.67,16530.87,16520.00,16520.69,294.73889
2023-01-01 00:15:00,16521.26,16537.73,16517.72,16534.94,481.18777
2023-01-01 00:20:00,16534.94,16540.66,16532.33,16535.54,309.53189
...,...,...,...,...,...
2024-11-01 23:40:00,69485.12,69515.16,69479.90,69489.83,27.34582
2024-11-01 23:45:00,69489.83,69499.99,69472.00,69484.00,36.44869
2024-11-01 23:50:00,69484.01,69560.00,69478.00,69540.00,33.85343


In [69]:
# Assuming 'data' is your DataFrame with 'close' and 'volume' columns
dataset['RSI'] = ta.rsi(dataset['close'], length=14)
# Adding the MA calculations
dataset['EMA_5'] = dataset['close'].ewm(span=5, adjust=False).mean()
dataset['EMA_30'] = dataset['close'].ewm(span=30, adjust=False).mean()
dataset['EMA_90'] = dataset['close'].ewm(span=90, adjust=False).mean()
#Volume 
dataset['volume_sma_3'] = dataset['volume'].rolling(window=3).mean()  # 3-day SMA for volume

In [70]:
# Calculate Bollinger Bands
bbands = ta.bbands(dataset['close'], length=20, std=2)

In [71]:
#Adding the limits of the Bollinger Bands
dataset['BB_upper'] = bbands['BBU_20_2.0']
dataset['BB_middle'] = bbands['BBM_20_2.0']
dataset['BB_lower'] = bbands['BBL_20_2.0']

In [72]:
# Step 2: Determine trend based on EMA alignment
def determine_trend(row):
    if (row['EMA_5'] > row['EMA_30'] > row['EMA_90']) & (row['close'] > row['BB_middle']) & (row['RSI'] < 70) & (row['volume'] > row['volume_sma_3']):
        return 'strong_upward'
    elif (row['EMA_5'] < row['EMA_30'] < row['EMA_90']) & (row['close'] < row['BB_middle']) & (row['RSI'] > 30):
        return 'strong_downward'
    elif (row['EMA_5'] > row['EMA_30'] < row['EMA_90']) & (row['RSI'] > 30) & (row['close'] < row['BB_middle']) & (row['volume'] < row['volume_sma_3']):
        return 'downward'
    elif (row['EMA_5'] < row['EMA_30'] > row['EMA_90']) & (row['RSI'] < 70) & (row['close'] > row['BB_lower']) & (row['volume'] > row['volume_sma_3']):
        return 'upward'
    else:
        return 'sideways'

In [73]:
dataset['trend'] = dataset.apply(determine_trend, axis=1)

In [74]:
# Step 3: Define actions and amounts based on the trend
def determine_action_amount(trend):
    if trend == 'strong_upward':
        return 'buy', 'all'
    elif trend == 'upward':
        return 'buy', 'half'
    elif trend == 'sideways':
        return 'hold', 'half'
    elif trend == 'downward':
        return 'sell', 'half'
    elif trend == 'strong_downward':
        return 'sell', 'all'
    else:
        return 'hold', 'none'

In [75]:
dataset[['action', 'amount']] = dataset['trend'].apply(lambda x: pd.Series(determine_action_amount(x)))

In [76]:
# Target columns (shifted to predict the next action)
dataset['target_trend'] = dataset['trend'].shift(-1)
dataset['target_action'] = dataset['action'].shift(-1)
dataset['target_amount'] = dataset['amount'].shift(-1)

In [77]:
dataset

,open,high,low,close,volume,RSI,EMA_5,EMA_30,EMA_90,volume_sma_3,BB_upper,BB_middle,BB_lower,trend,action,amount,target_trend,target_action,target_amount
timestamp,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,16541.77,16544.76,16527.51,16535.38,486.60903,NaN,16535.380000,16535.380000,16535.380000,NaN,NaN,NaN,NaN,sideways,hold,half,sideways,hold,half
2023-01-01 00:05:00,16534.91,16540.43,16522.55,16526.67,391.19043,NaN,16532.476667,16534.818065,16535.188571,NaN,NaN,NaN,NaN,sideways,hold,half,sideways,hold,half
2023-01-01 00:10:00,16526.67,16530.87,16520.00,16520.69,294.73889,NaN,16528.547778,16533.906576,16534.869922,390.846117,NaN,NaN,NaN,sideways,hold,half,sideways,hold,half
2023-01-01 00:15:00,16521.26,16537.73,16517.72,16534.94,481.18777,NaN,16530.678519,16533.973249,16534.871462,389.039030,NaN,NaN,NaN,sideways,hold,half,sideways,hold,half
2023-01-01 00:20:00,16534.94,16540.66,16532.33,16535.54,309.53189,NaN,16532.299012,16534.074330,16534.886155,361.819517,NaN,NaN,NaN,sideways,hold,half,sideways,hold,half
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-01 23:40:00,69485.12,69515.16,69479.90,69489.83,27.34582,57.351248,69499.354209,69397.646767,69503.901125,20.330643,69614.752757,69435.4080,69256.063243,sideways,hold,half,sideways,hold,half
2024-11-01 23:45:00,69489.83,69499.99,69472.00,69484.00,36.44869,56.777375,69494.236140,69403.217944,69503.463738,26.405213,69619.073497,69441.0075,69262.941503,sideways,hold,half,sideways,hold,half
2024-11-01 23:50:00,69484.01,69560.00,69478.00,69540.00,33.85343,60.831645,69509.490760,69412.042592,69504.266732,32.549313,69630.007115,69447.5075,69265.007885,sideways,hold,half,sideways,hold,half


In [78]:
# Drop rows with NaN values in the target
df2 = dataset.dropna().copy()

In [79]:
# Fit and transform the labels
df2['target_trend'] = le_trend.fit_transform(df2['target_trend'])
df2['target_action'] = le_action.fit_transform(df2['target_action'])
df2['target_amount'] = le_amount.fit_transform(df2['target_amount'])

In [80]:
# Define features and targets
X = df2[['EMA_5', 'EMA_30', 'EMA_90', 'volume', 'RSI', 'BB_upper', 'BB_middle', 'BB_lower', 'volume_sma_3']]
y_trend = df2['target_trend']
y_action = df2['target_action']
y_amount = df2['target_amount']

In [81]:
# Split the data
X_train_trend, X_test_trend, y_train_trend, y_test_trend = train_test_split(X, y_trend, test_size=0.15, random_state=101)
X_train_action, X_test_action, y_train_action, y_test_action = train_test_split(X, y_action, test_size=0.15, random_state=101)
X_train_amount, X_test_amount, y_train_amount, y_test_amount = train_test_split(X, y_amount, test_size=0.15, random_state=101)

In [82]:
# Train the XGBoost model for each target
model_trend = xgb.XGBClassifier(use_label_encoder=False)
model_action = xgb.XGBClassifier(use_label_encoder=False)
model_amount = xgb.XGBClassifier(use_label_encoder=False)

In [83]:
model_trend.fit(X_train_trend, y_train_trend)
model_action.fit(X_train_action, y_train_action)
model_amount.fit(X_train_amount, y_train_amount)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [84]:
# Predict and evaluate
y_pred_trend = model_trend.predict(X_test_trend)
y_pred_action = model_action.predict(X_test_action)
y_pred_amount = model_amount.predict(X_test_amount)

In [88]:
# Evaluation Trend
print("Trend accuracy: "+ str(accuracy_score(y_test_trend, y_pred_trend)))
print("Trend precision: "+ str(precision_score(y_test_trend, y_pred_trend, average="weighted")))
print("Trend recall: " + str(recall_score(y_test_trend, y_pred_trend, average="weighted")))
print( "Trend f1_score: " + str(f1_score(y_test_trend, y_pred_trend, average="weighted")))
print(confusion_matrix(y_test_trend, y_pred_trend))

Trend accuracy: 0.7061279414809192
Trend precision: 0.6728810900012195
Trend recall: 0.7061279414809192
Trend f1_score: 0.6449501629414325
[[    0   217    12     0     0]
 [    0 15522  1912    28    62]
 [    0  1657  4851     0     8]
 [    0  3073     5    40     0]
 [    0  1112   431     0    52]]


In [89]:
# Evaluation Action
print("Action accuracy: "+ str(accuracy_score(y_test_action, y_pred_action)))
print("Action precision: "+ str(precision_score(y_test_action, y_pred_action, average="weighted")))
print("Action recall: " + str(recall_score(y_test_action, y_pred_action, average="weighted")))
print( "Action f1_score: " + str(f1_score(y_test_action, y_pred_action, average="weighted")))
print(confusion_matrix(y_test_action, y_pred_action))

Action accuracy: 0.7064384790559658
Action precision: 0.6771265311536965
Action recall: 0.7064384790559658
Action f1_score: 0.6486600252525934
[[  101  4196   416]
 [   84 15480  1960]
 [    7  1845  4893]]


In [90]:
#Evaluation Amount
print("Amount accuracy: "+ str(accuracy_score(y_test_amount, y_pred_amount)))
print("Amount precision: "+ str(precision_score(y_test_amount, y_pred_amount, average="weighted")))
print("Amount recall: " + str(recall_score(y_test_amount, y_pred_amount, average="weighted")))
print( "Amount f1_score: " + str(f1_score(y_test_amount, y_pred_amount, average="weighted")))
print(confusion_matrix(y_test_amount, y_pred_amount))

Amount accuracy: 0.7548478365882272
Amount precision: 0.7475778591764384
Amount recall: 0.7548478365882272
Amount f1_score: 0.7374290689238862
[[ 4304  5330]
 [ 1775 17573]]


### Second Model with Parameter Optimization

In [38]:
# Split the data
X_train_trend_2, X_test_trend_2, y_train_trend_2, y_test_trend_2 = train_test_split(X, y_trend, test_size=0.2, random_state=101)
X_train_action_2, X_test_action_2, y_train_action_2, y_test_action_2 = train_test_split(X, y_action, test_size=0.2, random_state=101)
X_train_amount_2, X_test_amount_2, y_train_amount_2, y_test_amount_2 = train_test_split(X, y_amount, test_size=0.2, random_state=101)

In [91]:
# Parameter grid for optimization
param_grid = {
    'max_depth': [6.6,7],
    'learning_rate': [0.4, 0.5],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.8, 1.0],
    'reg_alpha': [0.3, 0.4],
    'reg_lambda': [0, 0.1],
    'min_child_weight':[0.5,0.6]
}

In [92]:
# Grid search for each target model
def optimize_model(X_train, y_train):
    xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
    grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    print(f"Best parameters found: {grid_search.best_params_}")
    return grid_search.best_estimator_

In [93]:
# Optimize models
print("Optimizing trend model...")
model_trend_2 = optimize_model(X_train_trend_2, y_train_trend_2)

Optimizing trend model...
Fitting 3 folds for each of 128 candidates, totalling 384 fits
Best parameters found: {'colsample_bytree': 1.0, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 0.6, 'reg_alpha': 0.3, 'reg_lambda': 0.1, 'subsample': 0.8}


In [94]:
print("Optimizing action model...")
model_action_2 = optimize_model(X_train_action_2, y_train_action_2)

Optimizing action model...
Fitting 3 folds for each of 128 candidates, totalling 384 fits
Best parameters found: {'colsample_bytree': 0.8, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 0.5, 'reg_alpha': 0.4, 'reg_lambda': 0, 'subsample': 0.8}


In [95]:
print("Optimizing amount model...")
model_amount_2 = optimize_model(X_train_amount_2, y_train_amount_2)

Optimizing amount model...
Fitting 3 folds for each of 128 candidates, totalling 384 fits
Best parameters found: {'colsample_bytree': 1.0, 'learning_rate': 0.4, 'max_depth': 7, 'min_child_weight': 0.6, 'reg_alpha': 0.4, 'reg_lambda': 0.1, 'subsample': 0.8}


In [96]:
# Evaluate the optimized models
y_pred_trend_2 = model_trend_2.predict(X_test_trend_2)
y_pred_action_2 = model_action_2.predict(X_test_action_2)
y_pred_amount_2 = model_amount_2.predict(X_test_amount_2)

In [97]:
print("Trend Accuracy:", accuracy_score(y_test_trend_2, y_pred_trend_2))
print("Action Accuracy:", accuracy_score(y_test_action_2, y_pred_action_2))
print("Amount Accuracy:", accuracy_score(y_test_amount_2, y_pred_amount_2))

Trend Accuracy: 0.7123041888068457
Action Accuracy: 0.7093368297564006
Amount Accuracy: 0.7623352425643503


In [98]:
# Evaluation Trend Optimized
print("Trend Optimized accuracy: "+ str(accuracy_score(y_test_trend_2, y_pred_trend_2)))
print("Trend Optimized precision: "+ str(precision_score(y_test_trend_2, y_pred_trend_2, average="weighted")))
print("Trend Optimized recall: " + str(recall_score(y_test_trend_2, y_pred_trend_2, average="weighted")))
print( "Trend Optimized f1_score: " + str(f1_score(y_test_trend_2, y_pred_trend_2, average="weighted")))
print(confusion_matrix(y_test_trend_2, y_pred_trend_2))

Trend Optimized accuracy: 0.7123041888068457
Trend Optimized precision: 0.6680697911582941
Trend Optimized recall: 0.7123041888068457
Trend Optimized f1_score: 0.6610922887986774
[[    1   213    14     1     0]
 [    6 15456  1735   173   154]
 [    2  1575  4910     2    27]
 [    0  2962     5   145     6]
 [    0  1148   315     0   132]]


In [99]:
# Evaluation Action Optimized
print("Action Optimized accuracy: "+ str(accuracy_score(y_test_action_2, y_pred_action_2)))
print("Action Optimized precision: "+ str(precision_score(y_test_action_2, y_pred_action_2, average="weighted")))
print("Action Optimized recall: " + str(recall_score(y_test_action_2, y_pred_action_2, average="weighted")))
print( "Action Optimized f1_score: " + str(f1_score(y_test_action_2, y_pred_action_2, average="weighted")))
print(confusion_matrix(y_test_action_2, y_pred_action_2))

Action Optimized accuracy: 0.7093368297564006
Action Optimized precision: 0.6662006726250781
Action Optimized recall: 0.7093368297564006
Action Optimized f1_score: 0.6592256324539756
[[  230  4116   367]
 [  292 15383  1849]
 [   26  1774  4945]]


In [100]:
#Evaluation Amount Optimized
print("Amount Optimized accuracy: "+ str(accuracy_score(y_test_amount_2, y_pred_amount_2)))
print("Amount Optimized precision: "+ str(precision_score(y_test_amount_2, y_pred_amount_2, average="weighted")))
print("Amount Optimized recall: " + str(recall_score(y_test_amount_2, y_pred_amount_2, average="weighted")))
print( "Amount Optimized f1_score: " + str(f1_score(y_test_amount_2, y_pred_amount_2, average="weighted")))
print(confusion_matrix(y_test_amount_2, y_pred_amount_2))

Amount Optimized accuracy: 0.7623352425643503
Amount Optimized precision: 0.7554772520233477
Amount Optimized recall: 0.7623352425643503
Amount Optimized f1_score: 0.7475577015361663
[[ 4575  5059]
 [ 1829 17519]]
